In [ ]:
import os
import geopandas as gpd
import ibis
from ibis import _
from cng.utils import *
from cng.h3 import *
import ibis.expr.datatypes as dt  
import re
from minio import Minio
import altair as alt

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
con.raw_sql("SET THREADS=100;")
set_secrets(con)

bucket = "public-election"
s3_prefix = "league_of_conservation_voters/members_of_congress"

lcv = con.read_csv(f's3://{bucket}/{s3_prefix}/**')

# Linear model w/ mixed effects

In [ ]:
# --- Panel regression with year + chamber fixed effects, clustered SEs by member ---
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# 1) Pull from ibis -> pandas
df = lcv.execute()  # or lcv.to_pandas() depending on your ibis version

# 2) Basic cleaning / variables
df["year"] = df["year"].astype(int)
df["year_score"] = pd.to_numeric(df["year_score"], errors="coerce")

# standardize party strings (LCV sometimes uses e.g. "R", "D", "Republican", "Democrat")
df["party_clean"] = (
    df["party"]
    .astype(str)
    .str.strip()
    .str.upper()
)

# Republican indicator (R = 1, else 0). If you want ONLY D vs R, filter below.
df["R"] = df["party_clean"].isin(["R", "REPUBLICAN"]).astype(int)

# Chamber FE needs to be categorical (and reasonably clean)
df["chamber"] = df["chamber"].astype(str).str.strip()

# Member identifier for clustering (best: stable ID; here: name+state+district+chamber)
# If you have a member ID field elsewhere, use that instead.
df["member_id"] = (
    df["first_name"].astype(str).str.strip().str.upper() + "_" +
    df["last_name"].astype(str).str.strip().str.upper() + "_" +
    df["state"].astype(str).str.strip().str.upper() + "_" +
    df["district"].astype(str).str.strip().str.upper() + "_" +
    df["chamber"].astype(str).str.strip().str.upper()
)

# Optional: restrict to just Democrats vs Republicans (recommended for clean interpretation)
df = df[df["party_clean"].isin(["D", "DEMOCRAT", "R", "REPUBLICAN"])].copy()

# Drop missing outcome / key fields
df = df.dropna(subset=["year_score", "year", "chamber", "member_id", "R"]).copy()

# 3) Fixed effects panel regression:
# Y_it = β0 + β1 R_it + γ_t (year FE) + δ_c (chamber FE) + ε_it
# Cluster SEs by member
model = smf.ols("year_score ~ R + C(year) + C(chamber)", data=df).fit(
    cov_type="cluster",
    cov_kwds={"groups": df["member_id"]}
)

print(model.summary())

# 4) One-sided hypothesis test: H_A: β1 < 0
beta1 = model.params["R"]
se1 = model.bse["R"]
t_stat = beta1 / se1

# statsmodels reports two-sided p-values. Convert to one-sided p-value for "less than 0".
p_two = model.pvalues["R"]
p_one = p_two / 2 if t_stat < 0 else 1 - (p_two / 2)

print("\nKey coefficient (Republican indicator):")
print(f"beta1 (R) = {beta1:.3f}")
print(f"SE(clustered by member) = {se1:.3f}")
print(f"t = {t_stat:.3f}")
print(f"one-sided p-value for H_A: beta1 < 0  =>  p = {p_one:.4g}")

# 5) (Optional) Nice, journal-friendly effect statement
# Difference in expected LCV score between Republicans and Democrats in same year & chamber
print(f"\nInterpretation: Republicans score about {beta1:.1f} points lower than Democrats on average (year+chamber FE).")


# Visualizing 

In [ ]:
## graphing utils 
colors = {
    "ind_yellow": "#ffbf00",
    "dem_blue": "#1b46c2",
    "rep_red": "#E81B23"
}

party_colors = alt.Scale(
    domain=["Democrat", "Republican","Independent"],
    # domain=[LABEL_D, LABEL_R],
    range=[colors["dem_blue"], colors["rep_red"], colors["ind_yellow"]],
)


def year_line(df, y, group, title, y_title, stat='percent'):
    legend = alt.Legend(
        title=None,
        labelFontSize=14,
        labelLimit=500,
        orient='top',
        direction='horizontal',
        offset=5
    )

    if stat == 'percent':
        y_axis = alt.Axis(format="%", labelFontSize=14, titleFontSize=18)
    elif stat == 'count':
        y_axis = alt.Axis(format="d", labelFontSize=14, titleFontSize=18)
    else:
        y_axis = alt.Axis(
            format="$,.0f",
            labelExpr="datum.value / 1000000",
            labelFontSize=14,
            titleFontSize=18,
        )

    x_axis = alt.Axis(
        labelFontSize=14,
        titleFontSize=18,
        labelPadding=4,
        titlePadding=10,
        labelExpr="(toNumber(datum.value) % 2 === 0) ? datum.value : ''"
    )

    return (
        alt.Chart(df, title=alt.TitleParams(text=title, fontSize=20, dy=-5))
        .mark_line(point=True)
        .encode(
            x=alt.X("year:O", title="Year", axis=x_axis),
            y=alt.Y(f"{y}:Q", title=y_title, axis=y_axis),
            color=alt.Color("party:N", scale=party_colors, legend=legend),
        )
        .properties(width=800, height=160)
    )

In [ ]:
df = (lcv.group_by('year','party')
    .agg(med_year_score=_.year_score.median()/100)
    .mutate(party=_.party.substitute({'I':"Independent","R":"Republican","D":"Democrat"}))
    .order_by("year").execute()
     )

chart = year_line(
    df,
    y="med_year_score",
    group="party",
    title="Median LCV Scores of Members of Congress by Party (1988–2025)",
    y_title="LCV Score (Median)",
    stat='percent'
)


# chart.save('lcv_party.png', ppi=200)
chart

# Specific votes

In [ ]:
MAIN_COLUMNS = {
"year": "bigint",
"chamber":"varchar",
"vote_title": "varchar",
"roll_call_vote_number":"bigint",
"vote_link":"varchar",
"votes_for": "bigint",
"votes_against": "bigint",
"not_voting": "bigint",
"pro_enviro_vote": "varchar",
"outcome": "varchar",
"id": "varchar",
}
s3_prefix = "league_of_conservation_voters/congressional_votes"
url = f's3://{bucket}/{s3_prefix}/**.csv'
all_votes=  con.read_csv(url,columns = MAIN_COLUMNS)

print(all_votes.select('id').distinct().count().execute())
print(all_votes.distinct().count().execute())

In [ ]:
url = f's3://{bucket}/{s3_prefix}/roll_call_votes/**.csv'
ind_df = con.read_csv(url).distinct()

member_year = (ind_df
    .mutate(member_id=_.first_name+'-'+_.last_name+'-'+_.state+'-'+_.chamber+'-'+_.district)
    .group_by('year','party','chamber','member_id')
    .agg(total_votes=_.count(),
         total_pro_votes=(_.vote_type=='pro').sum())
    .mutate(pro_rate=_.total_pro_votes/_.total_votes)
    .mutate(party=_.party.substitute({'I':"Independent","R":"Republican","D":"Democrat"}))
    .order_by("total_votes")   
     )

party_year = (
    member_year
    .group_by("year", "party")
    .agg(
        med_pro_rate=_.pro_rate.median(),
        n_members=_.member_id.nunique(),
        n_member_years=_.pro_rate.count(),
    )
    .order_by("year")
)


chart_cons_time = year_line(
    party_year,
    y="med_pro_rate",
    group="party",
    title=["Median Pro-Conservation Vote Rate by Party (Includes Absences)" ],
    y_title=["Pro-Conservation","Vote Rate (Median)"],
    stat="percent",
)

chart_cons_time.save('lcv_include_absences.png', ppi=300)
# chart_cons_time.save('lcv_exclude_absences.png', ppi=300)

chart_cons_time